# Clover fusion prediction 

In [3]:
#| default_exp 52-1-encoder-parallel-ramen-clover-with-input-concatenation-for-wikititles

In [4]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [5]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from scipy import sparse
from xcai.basics import *
from xcai.models.PPP0XX import DBT017

In [6]:
import xclib.evaluation.xc_metrics as xc_metrics

In [7]:
#| export
os.environ['WANDB_MODE'] = 'disabled'

In [8]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT']='xc-nlg_52-encoder-parallel-ramen-clover-with-input-concatenation-for-wikititles'

## Prediction

In [9]:
#| export
data_dir = '/home/aiscuser/scratch/datasets'

pkl_dir = f'{data_dir}/processed/'
pkl_file = f'{pkl_dir}/wikititles_data-metas_distilbert-base-uncased_rm_ramen-sal-encoder-parallel.pkl'

with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [10]:
#| export
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 32, True)

block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_hlk']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_hlk']

block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_hlk']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_hlk']

In [13]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/52-encoder-parallel-ramen-clover-with-input-concatenation-for-wikititles-1-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    output_concatenation_weight=1.0,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
)

In [15]:
#| export
output_dir = f"/home/aiscuser/scratch/Projects/xc_nlg/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [17]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT017.from_pretrained(mname, ig_tok=0, bsz=bsz, tn_targ=1000, margin=0.3, tau=0.1, 
                               n_negatives=5, apply_softmax=True, lw=0.01, m_lw=0.3, meta_prefix='cat', 
                               tie_word_embeddings=False)

In [18]:
#| export
trie = XCTrie.from_block(block)

  0%|          | 0/501070 [00:00<?, ?it/s]

In [57]:
#| export
train_dset = block.train.dset.sample(n=50_000, seed=50)
metric = PrecRecl(block.n_lbl, train_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

learn = XCLearner(model=model, args=args, trie=trie, train_dataset=block.train.dset, eval_dataset=train_dset,
                  data_collator=block.collator, compute_metrics=metric)

if __name__ == '__main__':
    mp.freeze_support()
    train_pred = learn.predict(train_dset)
    
display_metric(train_pred.metrics)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
#| export
pred_dir = f'{mname}/predictions/'
os.makedirs(pred_dir, exist_ok=True)
with open(f'{pred_dir}/train_predictions.pkl', 'wb') as file: pickle.dump(train_pred, file)

In [19]:
model.use_generation = False

In [20]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

learn = XCLearner(model=model, args=args, trie=trie, train_dataset=block.train.dset, eval_dataset=test_dset,
                  data_collator=block.collator, compute_metrics=metric)

if __name__ == '__main__':
    mp.freeze_support()
    test_pred = learn.predict(block.test.dset)
    
display_metric(test_pred.metrics)

[2024-06-10 20:33:09,167] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


  0%|          | 0/314 [00:00<?, ?it/s]

node-0:1191411:1191411 [0] NCCL INFO Bootstrap : Using eth0:10.13.60.54<0>
node-0:1191411:1191411 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:1191411:1191411 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:1191411:1191411 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:1191411:1219571 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:1191411:1219571 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:1191411:1219571 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node

node-0:1191411:1219570 [0] NCCL INFO Ring 4 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 5 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 6 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 7 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 8 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 9 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 10 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 11 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 12 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0] NCCL INFO Ring 13 : 1 -> 0 -> 1 comm 0x1d666260 nRanks 02 busId 100000
node-0:1191411:1219570 [0]

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,60.7882,36.8697,25.929,15.2137,60.7882,48.8842,46.1569,45.8621,36.3786,36.148,35.7916,37.5555,36.3786,38.2945,40.1427,42.9772,44.781,59.8771,64.5815,0.0668,1243.7766,630.132,0.394


In [ ]:
#| export
pred_dir = f'{mname}/predictions/'
os.makedirs(pred_dir, exist_ok=True)
with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file: pickle.dump(test_pred, file)

## Fusion

In [12]:
import numpy as np

In [10]:
pred_dir = f'{mname}/predictions/'

with open(f'{pred_dir}/train_predictions.pkl', 'rb') as f: 
    train_pred = pickle.load(f)
    
with open(f'{pred_dir}/test_predictions.pkl', 'rb') as f: 
    test_pred = pickle.load(f)

In [13]:
gen_pred,targ = get_output_sparse(**test_pred.gen_output, n_lbl=block.n_lbl)
repr_pred,_ = get_output_sparse(**test_pred.repr_output, n_lbl=block.n_lbl)
gen_pred.data = np.exp(gen_pred.data)

In [14]:
train_gen_pred,train_targ = get_output_sparse(**train_pred.gen_output, n_lbl=block.n_lbl)
train_repr_pred,_ = get_output_sparse(**train_pred.repr_output, n_lbl=block.n_lbl)
train_gen_pred.data = np.exp(train_gen_pred.data)

In [22]:
prop = xc_metrics.compute_inv_propesity(block.train.dset.data.data_lbl, A=0.55, B=1.5)
fuser = ScoreFusion(prop)

In [23]:
fuser.fit(train_gen_pred, train_repr_pred, train_targ)

In [30]:
pred = fuser.predict(gen_pred, repr_pred, beta=0.1)

In [31]:
output = {
    'targ_idx': test_pred.gen_output['targ_idx'],
    'targ_ptr': test_pred.gen_output['targ_ptr'],
    'pred_idx': torch.tensor(pred.indices),
    'pred_ptr': torch.tensor([q-p for p,q in zip(pred.indptr, pred.indptr[1:])]),
    'pred_score': torch.tensor(pred.data),
}

In [32]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])
m = metric(**output)
display_metric(m, remove_prefix=False)

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,32.3128,21.7133,16.4126,10.3124,32.3128,32.1152,33.1314,34.8998,25.7029,28.1331,30.4293,34.7173,25.7029,28.2914,30.0372,32.1681,39.5329,52.4273,55.1471


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,32.3128,21.7133,16.4126,10.3124,32.3128,32.1152,33.1314,34.8998,25.7029,28.1331,30.4293,34.7173,25.7029,28.2914,30.0372,32.1681,39.5329,52.4273,55.1471


In [52]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])
m = metric(**output)
display_metric(m, remove_prefix=False)

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,31.1399,20.7286,15.6277,9.8765,31.1399,30.8007,31.7445,33.5378,24.4896,26.6505,28.803,33.1025,24.4896,26.9292,28.598,30.7434,38.1122,52.0618,55.2864


In [55]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])
m = metric(**output)
display_metric(m, remove_prefix=False)

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,30.4053,20.2255,15.3097,9.7354,30.4053,30.135,31.1323,32.987,24.9653,26.6902,28.7182,32.9481,24.9653,27.0559,28.6494,30.7709,37.6288,51.9256,55.2399
